In [5]:
import numpy as np     #只需要下载numpy库即可
import random
import GridWorld_v2

In [6]:
gamma = 0.95   #折扣因子，越接近0越近视

rows = 5      #记得行数和列数这里要同步改
columns = 5

# gridworld = GridWorld_v2.GridWorld_v2(rows=rows, columns=columns, forbiddenAreaNums=8, targetNums=2, seed = 52,forbiddenAreaScore=-10)
# gridworld = GridWorld_v2.GridWorld_v2(desc = [".#",".T"])             #赵老师4-1的例子
# gridworld = GridWorld_v2.GridWorld_v2(desc = ["##.T","...#","...."])  #随便弄的例子
gridworld = GridWorld_v2.GridWorld_v2(forbiddenAreaScore=-10, score=1,desc = [".....",".##..","..#..",".#T#.",".#..."]) 
#gridworld = GridWorld_v2(forbiddenAreaScore=-10, score=1,desc = ["T."]) 
gridworld.show()


value = np.zeros(rows*columns)       #初始化可以任意，也可以全0
qtable = np.zeros((rows*columns,5))  #初始化，这里主要是初始化维数，里面的内容会被覆盖所以无所谓


# np.random.seed(50)
policy = np.eye(5)[np.random.randint(0,5,size=(rows*columns))] 
gridworld.showPolicy(policy)

⬜️⬜️⬜️⬜️⬜️
⬜️🚫🚫⬜️⬜️
⬜️⬜️🚫⬜️⬜️
⬜️🚫✅🚫⬜️
⬜️🚫⬜️⬜️⬜️
🔄🔄➡️⬆️⬅️
🔄⏩️⏬⬆️⬇️
⬇️⬇️🔄➡️⬇️
⬆️🔄✅⏬⬇️
🔄⏬⬇️⬆️🔄


In [7]:
from IPython.display import clear_output

In [10]:
# 通过采样的方法计算action value，model free的话意味着不知道整个gridworld的概率了，所以不能直接套贝尔曼方程迭代求解
#这个代码是非常难调的，很难调收敛
policy = np.eye(5)[np.random.randint(0,5,size=(rows*columns))]  # 随机初始化策略，使用独热编码表示每个状态的动作概率分布
gridworld.show()  # 显示网格世界
gridworld.showPolicy(policy)  # 显示当前策略
print("random policy")  # 打印提示信息，表示当前是随机策略
trajectorySteps = 5000  # 定义每个轨迹的步数
epsilon = 0.1  # 初始化探索率，原注释中提到也可设为0.2，这个是超参数，可以自行调整
qtable = np.zeros((rows*columns,5))  # 生成Q表，即动作价值表
num_episodes = 400  # 定义训练的回合数
epsilon_start = 0.3
epsilon_end = 0.05
epsilon_decay_rate = 0.995
for episode in range(num_episodes):  # 开始迭代每个回合
    #探索率指数衰减
    # if(epsilon > 0.001) :  # 如果探索率大于0.001
    #     epsilon -= 0.001  # 逐渐降低探索率
    # else:
    #     epsilon = 0.001  # 确保探索率不低于0.001
    epsilon = max(epsilon_end, epsilon_start * (epsilon_decay_rate ** episode))    
    p1 = 1-epsilon * (4/5)  # 计算选择最优动作的概率
    p0 = epsilon/5  # 计算选择非最优动作的概率
    # trajectorySteps = int(20+epsilon*1000)
    print("trajectorySteps",trajectorySteps)  # 打印当前轨迹步数
    print(f"epision:{epsilon}, p1:{p1}, p0:{p0}")  # 打印当前探索率、选择最优动作和非最优动作的概率
    
    d = {1:p1, 0:p0}  # 创建一个字典，用于将策略中的0和1映射到对应的概率
    policy_epsilon = np.vectorize(d.get)(policy)  # 将策略转换为epsilon-greedy策略，也就是原本是0和1，现在变为概率值
    
    i = random.randint(0,24)  # 随机选择初始状态
    j = random.randint(0,4)  # 随机选择初始动作
    cnt = [0 for i in range(25)]  # 初始化每个状态的访问次数计数器
    qtable_rewards = [[0 for j in range(5)] for i in range(rows * columns)]  # 初始化每个状态-动作对的累计奖励，全是0
    qtable_nums =    [[0 for j in range(5)] for i in range(rows * columns)]  # 初始化每个状态-动作对的访问次数，全是0
    Trajectory = gridworld.getTrajectoryScore(nowState=i, action=j, policy=policy_epsilon, steps=trajectorySteps)  # 获取轨迹及其得分
    clear_output(wait=True)  # 清除输出，等待新的输出
    # 注意这里的返回值是大小为(trajectorySteps+1)的元组列表，因为把第一个动作也加入进去了
    score = 0  # 初始化累计折扣奖励
    #下面的代码和5.2是完全一样的
    for k in range(trajectorySteps,-1,-1):  # 从轨迹的最后一步开始逆向遍历
        tmpstate, tmpaction, tmpscore, _, __  = Trajectory[k]  # 解包轨迹中的状态、动作和得分
        cnt[tmpstate] += 1  # 对应状态的访问次数加1
        score = score*gamma + tmpscore  # 计算累计折扣奖励，从后往前更新
        qtable_rewards[tmpstate][tmpaction] += score  # 对应状态-动作对的累计奖励加上当前累计折扣奖励
        qtable_nums[tmpstate][tmpaction] += 1  # 对应状态-动作对的访问次数加1
        qtable[tmpstate][tmpaction] = qtable_rewards[tmpstate][tmpaction] / qtable_nums[tmpstate][tmpaction]  # 更新Q表
    values = []  # 初始化状态价值列表
    for i in range(25):  # 遍历每个状态
        v = 0  # 初始化状态价值
        for j in range(5):  # 遍历每个动作
            v += policy_epsilon[i][j] * qtable[i][j]  # 计算状态价值
        values.append(v)  # 将状态价值添加到列表中
    print(np.array(values).reshape(5,5))  # 打印状态价值矩阵，调试使用
    
    # print(qvalue.reshape(5,5))
    gridworld.showPolicy(policy)
    print(np.array(values).mean())
    
    
    policy = np.eye(5)[np.argmax(qtable,axis=1)]  #qtable的最优值作为更新策略，并用独热码来表示
    policy_epsilon = np.vectorize(d.get)(policy)  #将策略转换为epsilon-greedy策略，也就是原本是0和1，现在变为概率值
        
    # print(np.array(cnt).reshape(5,5))


    


    

[[-1.02460146e+00 -1.60979007e+00 -1.62333622e+00 -1.85644409e-02
  -5.31532379e-02]
 [-1.89360382e+00 -1.67015049e+00 -3.25274144e+00 -2.75446365e-01
   9.88449804e-01]
 [-1.56405799e+00 -1.87495161e+01 -1.56340332e+02 -1.53894902e+02
  -5.02504903e+00]
 [ 4.26029157e+00 -1.45003892e+01 -3.93600859e+01 -4.08090285e+01
  -1.69810376e+01]
 [ 3.33324006e+00  1.71436042e+00 -1.05348551e+00 -1.25644710e+00
  -1.39568341e+01]]
🔄🔄⬅️⬇️⬅️
🔄⏪⏫️⬆️🔄
⬆️🔄🔄⬅️🔄
⬇️⏩️✅⏪⬇️
⬆️⏩️➡️⬅️➡️
-18.584648414024205


In [9]:

gridworld.showPolicy(policy)


  # 显示当前策略#打印状态价值的平均值 根据Q表编  # 将更新后的策略转换为epsilon-greedy策略

⬅️➡️➡️➡️🔄
➡️⏬⏬⬆️⬆️
⬇️⬅️⏩️⬆️⬆️
⬅️⏩️✅⏫️⬇️
⬆️⏩️🔄🔄⬅️
